In [1]:
import numpy as np 
from strawberryfields import Program, Engine
from strawberryfields.ops import Sgate, Interferometer
from strawberryfields.decompositions import williamson
from thewalrus.quantum import Amat, reduced_gaussian, Qmat
from thewalrus import hafnian
from thewalrus.quantum.fock_tensors import density_matrix
from scipy.stats import unitary_group
from scipy.special import factorial

In [2]:
M = 20 # number of modes
r = 0.3 # squeezing parameter

U = unitary_group.rvs(M) # generate Haar random unitary

In [3]:
#use strawberry fields to generate covariance matrix

prog = Program(M)
eng = Engine("gaussian")

with prog.context as q:
    # apply squeezing to every mode
    for mode in range(M):
        Sgate(r) | q[mode]
    # apply linear optical unitary
    Interferometer(U) | q
    
results = eng.run(prog)
cov = results.state.cov()
mu = results.state.means()

In [4]:
# calculate error bounds as a function of the cutoff 
# error measured by total variation distance to true distribution
# (borrowing code from https://github.com/XanaduAI/thewalrus/blob/8e856d1139b4b6427f3f1112e39db1aaca244b61/thewalrus/quantum/fock_tensors.py#L605)
max_cutoff = 8
bounds = np.zeros([max_cutoff+1])
for i in range(M):
    mu_red, cov_red = reduced_gaussian(mu, cov, [i])
    ps = np.real_if_close(np.diag(density_matrix(mu_red, cov_red, cutoff=max_cutoff+1)))
    bounds += 1 - np.cumsum(ps)
    
for i in range(max_cutoff+1):
    print(f'cutoff = {i+1:2d}, error bound = {bounds[i]:2.4}')

cutoff =  1, error bound = 1.633
cutoff =  2, error bound = 0.1936
cutoff =  3, error bound = 0.02419
cutoff =  4, error bound = 0.003497
cutoff =  5, error bound = 0.0005372
cutoff =  6, error bound = 8.869e-05
cutoff =  7, error bound = 1.527e-05
cutoff =  8, error bound = 2.729e-06
cutoff =  9, error bound = 5.001e-07


In [5]:
cutoff = 4 # seems like a fairly safe choice

In [6]:
# sample a photon number basis measurement from the state

s = []
detector_outcomes = []
rng = np.random.default_rng() # choose a psuedo-RNG to use
for mode in range(M):
    m = mode + 1 #number of modes needed for sample
    # find covariance of reduced state
    reduced_mu, reduced_cov = reduced_gaussian(cov=cov, mu=mu, modes=np.arange(m))
    # split into pure and classical parts
    D, S = williamson(reduced_cov) # cov = S @ D @ S.T
    T = S @ S.T
    W = S @ (D - np.eye(2 * m)) @ S.T
    # sample a vector from the classical state
    R = rng.multivariate_normal(reduced_mu, W)
    #construct B matrix
    q, p = R[:m], R[m:]
    alpha = np.concatenate([q + 1j * p, q - 1j * p])
    gamma = np.linalg.inv(Qmat(T)) @ alpha
    A = Amat(T)
    B = A[:m, :m]
    B[np.diag_indices(m)] = gamma[:m]
    
    prob_n = np.zeros(cutoff)
    for n in range(cutoff):
        repeats = s + [mode] * n
        B_s = B[np.ix_(repeats, repeats)]
        # find probability using the hafnian
        # we don't need the full normalisation term as we can just normalise afterwards
        prob_n[n] = abs(hafnian(B_s, loop=True))**2 / factorial(n) 
    # normalise the probabilities
    prob_n /= prob_n.sum()
    # sample
    outcome = rng.choice(range(cutoff), p=prob_n)
    # store and print results
    detector_outcomes.append(outcome)
    s += [mode] * outcome
    print(outcome, end='')
print(' - done')

00003000031000000001 - done
